In [1]:
import sys
sys.path.append('../')
from os import listdir
from os.path import isfile, join

import random
import json

import time
from utils import create_driver

from data_sources.google_scholar import scrape_publication_details, scrape_google_scholar_profile, save_google_scholar_profile, save_publications_details

In [2]:
alr_scraped = [f[15:-5] for f in listdir('./data') if isfile(join('./data', f)) and 'publications' not in f and 'google_scholar' in f]

In [3]:
with open('./data/individual_profiles.json', 'r') as f:
    ind_profile = json.load(f)
    
google_scholar_profile_url = []

for profile in ind_profile:
    if profile['google_scholar'] is not None and profile['full_name'].lower().replace(' ', '_') not in alr_scraped:
        google_scholar_profile_url.append([profile['full_name'], profile['google_scholar']])
len(google_scholar_profile_url)

34

In [4]:
google_scholar_profile_url[:3]

[['Anwitaman Datta',
  'https://scholar.google.com/citations?hl=en&user=VWi3_OIAAAAJ'],
 ['Bo An', 'https://scholar.google.com/citations?hl=en&user=PEEpuNwAAAAJ'],
 ['Cham Tat Jen',
  'https://scholar.google.com/citations?hl=en&user=Lx3X7W0AAAAJ']]

# Scraping Google Scholar Profile Page

In [5]:
driver = create_driver(debug=True)

for profile in google_scholar_profile_url:
    name, url = profile[0], profile[1]

    g_profile = scrape_google_scholar_profile(url, driver)
    save_google_scholar_profile(g_profile, output_filename=f'google_scholar_{name.lower().replace(" ","_")}.json')

    if random.uniform(0,1)>0.8:
        time.sleep(random.uniform(10,20))

    if random.uniform(0,1)>0.9:
        driver.quit()
        driver = create_driver(debug=False)

Error getting citation statistics at https://scholar.google.com/citations?hl=en&user=w7C16A8AAAAJ#d=gsc_md_hist&t=1697988635360
list index out of range
Error in scraping publications at https://scholar.google.com/citations?hl=en&user=w7C16A8AAAAJ#d=gsc_md_hist&t=1697988635360
Message: element click intercepted: Element is not clickable at point (459, 1822)
  (Session info: chrome=118.0.5993.96)
Stacktrace:
0   chromedriver                        0x00000001004ac510 chromedriver + 4310288
1   chromedriver                        0x00000001004a44bc chromedriver + 4277436
2   chromedriver                        0x00000001000d7b6c chromedriver + 293740
3   chromedriver                        0x0000000100123cb4 chromedriver + 605364
4   chromedriver                        0x0000000100121d7c chromedriver + 597372
5   chromedriver                        0x000000010011f9a4 chromedriver + 588196
6   chromedriver                        0x000000010011ea1c chromedriver + 584220
7   chromedriver     

In [6]:
driver.quit()

# Scraping Citation Information

In [10]:
alr_scraped_pub = [f[15:-18] for f in listdir('./data') if isfile(join('./data', f)) and 'publications' in f and 'google_scholar' in f]
alr_scraped = [f[15:-5] for f in listdir('./data') if isfile(join('./data', f)) and 'publications' not in f and 'google_scholar' in f]

not_yet_scraped = list(set(alr_scraped) - set(alr_scraped_pub))

with open('./data/individual_profiles.json', 'r') as f:
    ind_profile = json.load(f)
    
google_scholar_profile_url = []

for profile in ind_profile:
    if profile['google_scholar'] is not None and profile['full_name'].lower().replace(' ', '_') in not_yet_scraped:
        google_scholar_profile_url.append([profile['full_name'], profile['google_scholar']])
len(google_scholar_profile_url)


['long_cheng', 'li_boyang', 'dmitrii_ustiugov', 'lu_shijian', 'miao_chun_yan', 'lin_weisi', 'zhang_tianwei', 'cong_gao', 'yu_han', 'yeo_chai_kiat', 'erik_cambria', 'pan,_sinno_jialin', 'zhang_hanwang', 'liu_weichen', 'wentong_cai', 'joty_shafiq_rayhan', 'lam_kwok_yan', 'jagath_chandana_rajapakse', 'luu_anh_tuan', 'luo_jun', 'a_s_madhukumar', 'alexei_sourin', 'li_mo', 'ong_chin_ann', 'lim_wei_yang_bryan', 'arvind_easwaran', 'chng_eng_siong', 'sun_aixin', 'anupam_chattopadhyay', 'dusit_niyato', 'siyuan_liu', 'li_yi']


34

In [15]:
driver = create_driver(debug=True)

for j, profile in enumerate(google_scholar_profile_url[30:]):
    name, url = profile[0], profile[1]
    with open(f'./data/google_scholar_{name.lower().replace(" ","_")}.json', 'r') as f:
        data = json.load(f)
    print(f'{j}, {name}')
    details = {}
    try:
        for i, publication in enumerate(data['publications']):
            key = publication['title_link']
            driver.get(key)
            time.sleep(random.uniform(3,6))
            details[key] = scrape_publication_details(key, driver)
            if random.uniform(0,1)>0.75:
                time.sleep(random.uniform(5,10))
            if random.uniform(0,1)>0.9:
                driver.quit()
                driver = create_driver(debug=False)
            
            if i%10==0:
                save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
    except Exception as e:
        print(f'Stopped at {j} {name}')
        print(e)

        save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
    
    
    save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
    driver.quit()
    driver = create_driver(debug=False)

    time.sleep(random.uniform(3*60, 4*60))
driver.quit()

0, Wen Yonggang
1, Zhang Jie
2, Zhao Jun
3, Zheng Jianmin


# Re running any publications with error

In [20]:
from striprtf.striprtf import rtf_to_text

with open("problem_google_scholar_publication.rtf") as infile:
    content = infile.read()
    text = rtf_to_text(content)
text = text.split('\n')
faulty_url = []
for t in text:
    if not t.isnumeric() and t!='':
        faulty_url.append(t.split(' at ')[1])

In [ ]:
driver = create_driver(debug=True)

for j, profile in enumerate(google_scholar_profile_url[1:]):
    name, url = profile[0], profile[1]
    with open(f'./data/google_scholar_{name.lower().replace(" ","_")}.json', 'r') as f:
        data = json.load(f)
    details = {}
    print(len(faulty_url))
    for i, publication in enumerate(data['publications']):
        key = publication['title_link']
        if key in faulty_url:
            driver.get(key)
            time.sleep(random.uniform(3,6))
            details[key] = scrape_publication_details(key, driver)
            if random.uniform(0,1)>0.75:
                time.sleep(random.uniform(5,10))
            if random.uniform(0,1)>0.9:
                driver.quit()
                driver = create_driver(debug=False)
            faulty_url.remove(key)

        save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')

    if len(faulty_url)==0:
        break
driver.quit()